# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/psuresh/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/psuresh/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/psuresh/Documents/2025AIEngineer/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/psuresh/Documents/2025AIEngineer/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/psuresh/Documents/2025AIEngineer/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [9]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '672a49'. Skipping!
Property 'summary' already exists in node '16e765'. Skipping!
Property 'summary' already exists in node 'd1eb01'. Skipping!
Property 'summary' already exists in node '1d51d5'. Skipping!
Property 'summary' already exists in node '1b4d73'. Skipping!
Property 'summary' already exists in node 'f15565'. Skipping!
Property 'summary' already exists in node 'de5b28'. Skipping!
Property 'summary' already exists in node 'c0ea18'. Skipping!
Property 'summary' already exists in node '61bfd9'. Skipping!
Property 'summary' already exists in node '59c506'. Skipping!
Property 'summary' already exists in node 'cafa93'. Skipping!
Property 'summary' already exists in node '77af4e'. Skipping!
Property 'summary' already exists in node 'e2ab91'. Skipping!
Property 'summary' already exists in node '03c144'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '672a49'. Skipping!
Property 'summary_embedding' already exists in node '16e765'. Skipping!
Property 'summary_embedding' already exists in node '59c506'. Skipping!
Property 'summary_embedding' already exists in node '1b4d73'. Skipping!
Property 'summary_embedding' already exists in node '77af4e'. Skipping!
Property 'summary_embedding' already exists in node 'f15565'. Skipping!
Property 'summary_embedding' already exists in node 'd1eb01'. Skipping!
Property 'summary_embedding' already exists in node 'e2ab91'. Skipping!
Property 'summary_embedding' already exists in node 'cafa93'. Skipping!
Property 'summary_embedding' already exists in node 'c0ea18'. Skipping!
Property 'summary_embedding' already exists in node '03c144'. Skipping!
Property 'summary_embedding' already exists in node '1d51d5'. Skipping!
Property 'summary_embedding' already exists in node '61bfd9'. Skipping!
Property 'summary_embedding' already exists in node 'de5b28'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 478)

We can save and load our knowledge graphs as follows.

In [10]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 478)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [12]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


##### ✅ Answer::
+ SingleHopSpecificQuerySynthesizer
This refers to a type of query synthesizer that generates queries requiring information from a single "hop" or source of information. 

+ MultiHopAbstractQuerySynthesizer
This synthesizer generates queries that require information from multiple hops or sources and then demand an abstract or high-level answer summarizing the retrieved information. It involves more complex reasoning and may require the LLM to synthesize information across different data points.

+ MultiHopSpecificQuerySynthesizer
Similar to the MultiHopAbstractQuerySynthesizer, this synthesizer generates queries requiring information from multiple sources but aims for a specific, factual answer rather than a generalized summary. 

Finally, we can use our `TestSetGenerator` to generate our testset!

In [13]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Whaat are the diffrent academic years for the ...,"[Chapter 1 Academic Years, Academic Calendars,...",Chapter 1 explains that each eligible program ...,single_hop_specifc_query_synthesizer
1,What is 34 CFR 668.3(a) about in terms of acad...,[Regulatory Citations Academic year minimums: ...,Regulatory Citations Academic year minimums: 3...,single_hop_specifc_query_synthesizer
2,What is the significance of Volume 8 in the co...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a standard term ...,single_hop_specifc_query_synthesizer
3,Is FWS a payment period?,[Non-Term Characteristics A program that measu...,The payment period is applicable to all Title ...,single_hop_specifc_query_synthesizer
4,How does completing additional clock hours or ...,[both the credit or clock hours and the weeks ...,Completing additional weeks of instructional t...,single_hop_specifc_query_synthesizer
5,How does the inclusion of clinical work in sta...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The inclusion of clinical work in standard ter...,multi_hop_abstract_query_synthesizer
6,How does 34 CFR 668.3(a) relate to the academi...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",34 CFR 668.3(a) specifies the minimum requirem...,multi_hop_abstract_query_synthesizer
7,How do the different academic year definitions...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The context explains that each eligible progra...,multi_hop_abstract_query_synthesizer
8,Appendix A and B disbursement rules how they r...,[<1-hop>\n\nboth the credit or clock hours and...,Appendix A explains how disbursement timing is...,multi_hop_specific_query_synthesizer
9,How do Volume 2 and Volume 8 relate to disburs...,[<1-hop>\n\nboth the credit or clock hours and...,Volume 2 discusses the timing of disbursements...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '337bf9'. Skipping!
Property 'summary' already exists in node '86732b'. Skipping!
Property 'summary' already exists in node '50b15c'. Skipping!
Property 'summary' already exists in node 'c870d6'. Skipping!
Property 'summary' already exists in node 'a7b355'. Skipping!
Property 'summary' already exists in node '631f59'. Skipping!
Property 'summary' already exists in node 'f39c62'. Skipping!
Property 'summary' already exists in node '086c97'. Skipping!
Property 'summary' already exists in node '3882e3'. Skipping!
Property 'summary' already exists in node '7dc847'. Skipping!
Property 'summary' already exists in node '2ccc25'. Skipping!
Property 'summary' already exists in node '0b3edc'. Skipping!
Property 'summary' already exists in node '3aa00b'. Skipping!
Property 'summary' already exists in node 'c208b9'. Skipping!


...

Property 'summary_embedding' already exists in node 'c208b9'. Skipping!
Property 'summary_embedding' already exists in node 'f39c62'. Skipping!
Property 'summary_embedding' already exists in node '7dc847'. Skipping!
Property 'summary_embedding' already exists in node '50b15c'. Skipping!
Property 'summary_embedding' already exists in node '086c97'. Skipping!
Property 'summary_embedding' already exists in node '337bf9'. Skipping!
Property 'summary_embedding' already exists in node 'a7b355'. Skipping!
Property 'summary_embedding' already exists in node '2ccc25'. Skipping!
Property 'summary_embedding' already exists in node 'c870d6'. Skipping!
Property 'summary_embedding' already exists in node '0b3edc'. Skipping!
Property 'summary_embedding' already exists in node '631f59'. Skipping!
Property 'summary_embedding' already exists in node '86732b'. Skipping!
Property 'summary_embedding' already exists in node '3aa00b'. Skipping!
Property 'summary_embedding' already exists in node '3882e3'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [15]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is Volume 2 in the context of academic ye...,"[Chapter 1 Academic Years, Academic Calendars,...",The provided context does not specify what Vol...,single_hop_specifc_query_synthesizer
1,What is 34 CFR 668.3(b) about?,[Regulatory Citations Academic year minimums: ...,34 CFR 668.3(b) relates to weeks of instructio...,single_hop_specifc_query_synthesizer
2,What defines a Standard Term in relation to cl...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a Standard Term ...,single_hop_specifc_query_synthesizer
3,How does Title IV regulation define and regula...,[Non-Term Characteristics A program that measu...,"According to the context, Title IV programs ge...",single_hop_specifc_query_synthesizer
4,What are the program requirements for academic...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The program requirements specify that every el...,multi_hop_abstract_query_synthesizer
5,how accelerated progression effects disburseme...,[<1-hop>\n\nboth the credit or clock hours and...,accelerated progression in clock-hour or non-t...,multi_hop_abstract_query_synthesizer
6,Is flexiblty in term clasification for clinika...,[<1-hop>\n\nInclusion of Clinical Work in a St...,"Yes, the context explains that clinical work c...",multi_hop_abstract_query_synthesizer
7,How does the measurement of academic progress ...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The measurement of academic progress in relati...,multi_hop_abstract_query_synthesizer
8,How do Volume 2 and Volume 8 address the inclu...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",Volume 2 discusses the requirements for defini...,multi_hop_specific_query_synthesizer
9,Appendix A and B disbursement rules how they r...,[<1-hop>\n\nboth the credit or clock hours and...,Appendix A explains that disbursement of Pell ...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [16]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [17]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [18]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [20]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [21]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [22]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [23]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [24]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [25]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [26]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available include:\n\n- Direct Subsidized Loans  \n- Direct Unsubsidized Loans  \n- Direct PLUS Loans (student Federal PLUS Loans and parent Direct PLUS Loans)  \n- Subsidized and Unsubsidized Federal Stafford Loans (made under the FFEL Program before July 1, 2010)  \n- Federal SLS Loans  \n- Federal PLUS Loans (made under the FFEL Program before July 1, 2010)  \n\nNote: New loans under the FFEL Program are no longer made since July 1, 2010. Direct Subsidized Loans are available only to undergraduate students. Graduate or professional students are eligible only for Direct Unsubsidized Loans.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [27]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [28]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.
##### ✅ Answer:


- `qa_evaluator`: Evaluating the question and answer response
- `labeled_helpfulness_evaluator`: This evaluator determines if the AI's response is helpful by comparing it against what the correct answer should be, rather than just evaluating it in isolation.
- `empathy_evaluator`: This is evaluating if the user is feeling heard

## LangSmith Evaluation

In [29]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'earnest-range-74' at:
https://smith.langchain.com/o/ac6791eb-67c7-4936-b727-03a31f82046f/datasets/f97a93a3-5dfa-4a1f-8cbb-97726448c1db/compare?selectedSessions=11d3ff56-d23d-4d65-bf9b-c8ccd039c61b




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Volume 2 and Volume 8 relate to the req...,"Based on the provided context, Volume 8 discus...",None,Volume 2 details the requirements for defining...,0,0,0,5.943909,ac313632-d6af-41fc-8da7-f56bec516616,27c68fac-70d6-4a1f-8371-d98e55fb1b5d
1,How do Volumes 7 and 8 relate to the disbursem...,Volumes 7 and 8 provide guidance related to di...,None,Volume 8 discusses the timing of disbursements...,1,0,0,3.520843,3e3ebf8f-ec3e-4bef-9a8f-5f4463fcdfc3,9bde9c71-93ec-4073-b5f0-37ac7098123a
2,Appendix A and B disbursement rules how they r...,"Based on the provided context, Appendix B in V...",None,Appendix A explains that disbursement of Pell ...,0,0,0,4.226466,e836ce00-2e01-4781-88ac-213f528a45be,d2de9712-5e81-47d7-8729-46bf56912640
3,How do Volume 2 and Volume 8 address the inclu...,"Based on the provided context, the following c...",None,Volume 2 discusses the requirements for defini...,1,1,0,17.730750,d35c5149-84ef-49f1-97ce-6a62ba1e3f3c,441206c7-7d40-45cf-b3b5-9af8a94c97dc
4,How does the measurement of academic progress ...,Based on the provided context:\n\nAcademic pro...,None,The measurement of academic progress in relati...,1,1,0,14.011027,0e680c61-20ec-4165-b641-84d93db530c5,1004a92c-fe0a-45c2-b7d9-2ace2d80722a
5,Is flexiblty in term clasification for clinika...,"Based on the provided context, flexibility in ...",None,"Yes, the context explains that clinical work c...",1,1,0,4.641004,5fd9bdaa-ff6c-434f-bf0c-d72d108ef00c,8a33c2a9-59e1-4eaf-aa36-f02bf7c8a911
6,how accelerated progression effects disburseme...,Based on the provided context:\n\n**Effect of ...,None,accelerated progression in clock-hour or non-t...,1,0,0,6.940483,2fd4c40c-7206-4be7-82ba-47f55e704d03,b3cb6c5d-d87b-4119-bb65-9fea3d27e46f
7,What are the program requirements for academic...,"Based on the provided context, the program req...",None,The program requirements specify that every el...,1,1,0,5.787797,007202a9-e198-4742-8d58-ca9456d8da8f,314c73e9-b624-4599-a3c0-aa4eaac03ada
8,How does Title IV regulation define and regula...,Title IV regulations define and regulate payme...,None,"According to the context, Title IV programs ge...",1,1,0,16.261706,61784484-3932-457b-ba82-d37e063d5e93,715f2162-5261-4b7a-be44-bb4fc7e71aa7
9,What defines a Standard Term in relation to cl...,A Standard Term may include periods of clinica...,None,Inclusion of clinical work in a Standard Term ...,1,0,0,3.555258,d27afff4-3cdb-4d7a-8cfe-f40be9ebfbb0,e8a8c4b2-c3d2-4756-9658-d3d6e8f5982b


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [30]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [31]:
rag_documents = docs

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

##### ✅ Answer:
Smaller chunks: 

These can capture more fine-grained details, leading to potentially higher precision in retrieval for specific queries. However, overly small chunks might lack the necessary context to fully understand the meaning of the text or to generate coherent responses.


Larger chunks:

 These provide a broader context, ensuring that more related information is available to the LLM. But excessively large chunks can dilute the relevance of a specific query, as they may contain irrelevant information or multiple distinct ideas. This can make it harder for the LLM to pinpoint the most relevant information and potentially lead to less accurate or focused responses. 

Modifying the chunk_size directly impacts the amount of context available to the LLM and the efficiency of the retrieval process, ultimately affecting the overall performance, relevance, and accuracy of your application's output

In [33]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

##### ✅ Answer:

Modifying and improving our embedding model directly impacts how well the RAG system understands and retrieves information, which in turn dictates the quality and relevance of the generated responses.

In [34]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [35]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [36]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [37]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question. Based on the information provided, there are several types of loans available for students and their parents to help with educational costs:\n\n1. **Direct Subsidized Loans** – These are loans for students with financial need, and the amount you can borrow is based on your cost of attendance (COA) minus your expected family contribution and other financial aid. These loans do not accrue interest while you’re in school.\n\n2. **Direct Unsubsidized Loans** – Available to both dependent and independent students, these loans are not based on financial need and do accrue interest while you are in school.\n\n3. **Direct PLUS Loans** – These loans are available to parents of dependent students (assuming eligibility) and to graduate or professional students themselves. They can cover the COA minus other financial aid received and do not have a fixed borrowing limit but cannot exceed the student’s COA minus aid.\n\nAdditionally, if a dependent student’s parent is u

Finally, we can evaluate the new chain on the same test set!

In [38]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'enchanted-insurance-28' at:
https://smith.langchain.com/o/ac6791eb-67c7-4936-b727-03a31f82046f/datasets/f97a93a3-5dfa-4a1f-8cbb-97726448c1db/compare?selectedSessions=78346d5a-0e01-491f-b79a-2f71d0721d81




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Volume 2 and Volume 8 relate to the req...,Thank you for your thoughtful question. Based ...,None,Volume 2 details the requirements for defining...,1,0,1,8.127070,ac313632-d6af-41fc-8da7-f56bec516616,e909e0e3-850b-4566-800b-cc5561a080c6
1,How do Volumes 7 and 8 relate to the disbursem...,Thank you for your thoughtful question. Based ...,None,Volume 8 discusses the timing of disbursements...,0,0,1,6.175536,3e3ebf8f-ec3e-4bef-9a8f-5f4463fcdfc3,f4f02179-36d2-4057-b249-87e7ee051f53
2,Appendix A and B disbursement rules how they r...,Thank you for your thoughtful question. Based ...,None,Appendix A explains that disbursement of Pell ...,0,0,1,7.388686,e836ce00-2e01-4781-88ac-213f528a45be,9320452e-d5a3-49c5-add7-53fd465eede6
3,How do Volume 2 and Volume 8 address the inclu...,Thank you for your thoughtful question. I see ...,None,Volume 2 discusses the requirements for defini...,0,0,1,9.055874,d35c5149-84ef-49f1-97ce-6a62ba1e3f3c,3fb087d1-2576-484f-bb22-2c8169ab58d8
4,How does the measurement of academic progress ...,Thank you for your thoughtful question. I unde...,None,The measurement of academic progress in relati...,1,1,1,7.537332,0e680c61-20ec-4165-b641-84d93db530c5,e5343191-ac2e-43bf-834e-b2899f37a0d6
5,Is flexiblty in term clasification for clinika...,Thank you for your thoughtful question. Based ...,None,"Yes, the context explains that clinical work c...",1,1,1,4.045074,5fd9bdaa-ff6c-434f-bf0c-d72d108ef00c,42d9843a-5b1e-45dc-804d-9a248f558069
6,how accelerated progression effects disburseme...,Thank you for your thoughtful question. I unde...,None,accelerated progression in clock-hour or non-t...,1,0,1,9.102923,2fd4c40c-7206-4be7-82ba-47f55e704d03,8f3defaa-1bf5-498c-8a0e-c190e54c9828
7,What are the program requirements for academic...,Thank you for your thoughtful question about p...,None,The program requirements specify that every el...,1,1,1,9.566794,007202a9-e198-4742-8d58-ca9456d8da8f,f2192d3b-9fb2-4514-a5ed-04633f2c976d
8,How does Title IV regulation define and regula...,Thank you for your thoughtful question. I unde...,None,"According to the context, Title IV programs ge...",1,1,1,7.984147,61784484-3932-457b-ba82-d37e063d5e93,f0437910-c4a0-4506-9cb4-a7f65ec982fd
9,What defines a Standard Term in relation to cl...,Thank you for your thoughtful question—it’s cl...,None,Inclusion of clinical work in a Standard Term ...,1,1,1,6.400041,d27afff4-3cdb-4d7a-8cfe-f40be9ebfbb0,eb45c099-39a6-4063-8f9f-ff707795d931


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.